In [32]:
# -*- coding: utf-8 -*-
"""
Created on Wed Aug  5 20:31:11 2020

@author: REZA
"""



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, GRU, LSTM, RNN

#from keras.models import Sequential, model_from_json
#from keras.layers import Conv1D,Conv2D,MaxPooling1D,Flatten,Dense,Dropout,BatchNormalization, GRU, LSTM, RNN
from tensorflow.keras import regularizers as reg

import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit, cross_val_score
from mne import Epochs,pick_types,events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws,read_raw_edf
from mne.datasets import eegbci


from sklearn.model_selection import train_test_split


from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time


In [35]:

#%%
def cnn(hp,conv_layers=3,filter_size=3, fc_layers=2,fc_sizes=(512,256),
        dropout=0.5,pool_size=2,init='he_uniform',act='relu',optim='adam',
        reg = reg.l2(0.05)):

    classifier = Sequential()
    
    classifier.add(Conv1D(hp.Int("input_units",min_value=32,max_value=512,step=32), filter_size, input_shape = X_train.shape[1:],
                              activation = act,kernel_initializer=init,kernel_regularizer=reg))
#    classifier.add(BatchNormalization())
    classifier.add(MaxPooling1D(pool_size=2))
    
    for i in range(hp.Int("n_layers",1,3)):
        classifier.add(Conv1D(hp.Int(f"Conv_{i}_units",min_value=32,max_value=256,step=32),3))
        classifier.add(Activation('relu'))
        
    classifier.add(Flatten())
    
    for j in range(fc_layers):
        classifier.add(Dense(fc_sizes[j], activation = act,kernel_initializer=init,kernel_regularizer=reg))
        classifier.add(Dropout(dropout))
        
    classifier.add(Dense(4, activation = 'softmax',kernel_initializer=init))
    classifier.compile(optimizer = optim, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return classifier
    

In [3]:




#%%
subject=1
runs=[6,10,14]
raw_fnames=eegbci.load_data(subject,runs)
raw=concatenate_raws([read_raw_edf(f,preload=True) for f in raw_fnames])

eegbci.standardize(raw)
#create 10-10 system
montage=make_standard_montage('standard_1005')
raw.set_montage(montage)

raw.filter(6,13)
events,_=events_from_annotations(raw,event_id=dict(T1=0,T2=1))
#0: left, 1:right
picks=pick_types(raw.info,meg=False,eeg=True,stim=False,eog=False,exclude='bads')

tmin,tmax=-1,4
epochs=Epochs(raw,events,None,tmin,tmax,proj=True,picks=picks,
             baseline=None,preload=True)
#epochs_train=epochs.copy().crop(0,2)
epochs_train=epochs.copy().crop(0,2)
labels=epochs.events[:,-1]

epochs_data=epochs.get_data()
epochs_data_train=epochs_train.get_data()


X_train,X_test,y_train,y_test = train_test_split(
        epochs_data_train, labels, test_size=0.2, random_state=42)



Extracting EDF parameters from C:\Users\REZA\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\REZA\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\REZA\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 6 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming 

In [4]:
classifier=cnn()
classifier.fit(X_train, y_train, validation_data=(X_test,y_test),epochs=10,batch_size=64)   

history=cnn(conv_layers=2,conv_sizes=(64,64),fc_layers=3,fc_sizes=(1024,512,256))


TypeError: cnn() missing 1 required positional argument: 'hp'

In [36]:
#%%
tuner=RandomSearch(cnn,
                   objective='val_accuracy',
                   max_trials=1,
                   executions_per_trial=1)

tuner.search(x=X_train,
             y=y_train,
            epochs=20,
            batch_size=64,
            validation_data=(X_test,y_test))


Epoch 1/20
1/1 [==============================] - ETA: 0s - loss: 123.0617 - accuracy: 0.50 - 1s 796ms/step - loss: 123.0617 - accuracy: 0.5000 - val_loss: 115.7835 - val_accuracy: 0.3333
Epoch 2/20
1/1 [==============================] - ETA: 0s - loss: 115.7798 - accuracy: 0.58 - 0s 30ms/step - loss: 115.7798 - accuracy: 0.5833 - val_loss: 108.8846 - val_accuracy: 0.3333
Epoch 3/20
1/1 [==============================] - ETA: 0s - loss: 108.8770 - accuracy: 0.61 - 0s 32ms/step - loss: 108.8770 - accuracy: 0.6111 - val_loss: 102.3621 - val_accuracy: 0.3333
Epoch 4/20
1/1 [==============================] - ETA: 0s - loss: 102.3543 - accuracy: 0.58 - 0s 28ms/step - loss: 102.3543 - accuracy: 0.5833 - val_loss: 96.2013 - val_accuracy: 0.3333
Epoch 5/20
1/1 [==============================] - ETA: 0s - loss: 96.1812 - accuracy: 0.583 - 0s 27ms/step - loss: 96.1812 - accuracy: 0.5833 - val_loss: 90.3865 - val_accuracy: 0.3333
Epoch 6/20
1/1 [==============================] - ETA: 0s - loss: 9

INFO:tensorflow:Oracle triggered exit
